**ETL review**

In [4]:
import pandas as pd

Para poder leer el archivo de reseñas de Yelp, tuve que dividirlo en fragmentos más pequeños, ya que su tamaño era considerable y no quería sobrecargar de los recursos. Esta estrategia de leer el archivo por partes me permitió gestionar de manera eficiente los recursos de memoria disponibles


In [ ]:
chunk_size = 100000  # Ajusta el tamaño del trozo según sea necesario
chunks = []

for chunk in pd.read_json("/content/drive/MyDrive/yelp/Copia de review.json", lines=True, chunksize=chunk_size):
    chunks.append(chunk)

df_review = pd.concat(chunks, ignore_index=True)

Después de la lectura inicial, dividiremos los datos en archivos CSV para un proceso eficiente de Extracción, Transformación y Carga (ETL). Cruzaremos estos archivos con la base de negocios de Florida, enfocándonos en las reseñas vinculadas a esos negocios. Así, obtendremos un conjunto depurado de reseñas de negocios en Florida para análisis específicos.



In [ ]:
# Dividir el DataFrame en lotes más pequeños
batch_size = len(df_review) // 6
df_batches = [df_review[i:i+batch_size] for i in range(0, len(df_review), batch_size)]

# Guardar cada lote en un archivo CSV
for i, batch in enumerate(df_batches):
    batch.to_csv(f'/content/drive/MyDrive/yelp/review_yelp/review_{i+1}.csv', index=False)


Leemos los archivos csv creados

In [ ]:
df_review1 = pd.read_csv("/content/drive/MyDrive/yelp/review_yelp/review_1.csv")
df_review2 = pd.read_csv("/content/drive/MyDrive/yelp/review_yelp/review_2.csv")
df_review3 = pd.read_csv("/content/drive/MyDrive/yelp/review_yelp/review_3.csv")
df_review4 = pd.read_csv("/content/drive/MyDrive/yelp/review_yelp/review_4.csv")
df_review5 = pd.read_csv("/content/drive/MyDrive/yelp/review_yelp/review_5.csv")
df_review6 = pd.read_csv("/content/drive/MyDrive/yelp/review_yelp/review_6.csv")
df_review7 = pd.read_csv("/content/drive/MyDrive/yelp/review_yelp/review_7.csv")

In [9]:
df_review1.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30


Leemos el archivo business con el cual realizaremos el merge


In [10]:
df_business = pd.read_csv("/content/drive/MyDrive/Otros/yelp_business_florida_final.csv")

<ipython-input-10-28ac4b79d0cc>:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_business = pd.read_csv("/content/drive/MyDrive/Otros/yelp_business_florida_final.csv")


In [11]:
df_business.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Restaurants,Sandwiches,Shopping,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,FL,33707.0,27.766590,-82.732983,3.5,5,...,0.0,0.0,0.0,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,NaN,NaN
1,jaxMSoInw8Poo3XeMJt8lQ,Adams Dental,15 N Missouri Ave,Clearwater,FL,33755.0,27.966235,-82.787412,5.0,10,...,1.0,0.0,0.0,7:30-15:30,7:30-15:30,7:30-15:30,7:30-15:30,NaN,NaN,NaN
2,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771.0,27.916116,-82.760461,4.5,100,...,0.0,0.0,0.0,10:0-18:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,NaN


Se crea la funcion para realizar el merge de todas las partes del dataset de reviews



In [ ]:
def merge_reviews_with_business(df_review1, chunk_size=10000):
    review_chunks = [df_review1[i:i + chunk_size] for i in range(0, len(df_review1), chunk_size)]

    merged_chunks = []

    for chunk in review_chunks:
        merged_chunk = pd.merge(chunk, df_business, on='business_id')
        merged_chunks.append(merged_chunk)

    merged_df = pd.concat(merged_chunks)

    return merged_df

In [ ]:
merged_df_review1 = merge_reviews_with_business(df_review1)
merged_df_review2 = merge_reviews_with_business(df_review2)
merged_df_review3 = merge_reviews_with_business(df_review3)
merged_df_review4 = merge_reviews_with_business(df_review4)
merged_df_review5 = merge_reviews_with_business(df_review5)
merged_df_review6 = merge_reviews_with_business(df_review6)
merged_df_review7 = merge_reviews_with_business(df_review7)

Concatenamos todos los df resultantes


In [ ]:
dataframes = [merged_df_review1, merged_df_review2, merged_df_review3,
              merged_df_review4, merged_df_review5, merged_df_review6, merged_df_review7]

concatenated_df = pd.concat(dataframes, ignore_index=True)

Filtro para dejar solo las columnas de el dataset de reviews

In [ ]:
concatenated_df = concatenated_df[["review_id",	"user_id",	"business_id",	"stars_x",	"useful",	"funny",	"cool",	"text",	"date"]]

In [13]:
concatenated_df.head(3)

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date
0,rGQRf8UafX7OTlMNN19I8A,1WHRWwQmZOZDAhp2Qyny4g,uMvVYRgGNXf5boolA9HXTw,5,2,0,0,My experience with Shalimar was nothing but wo...,2015-06-21 14:48:06
1,OAhBYw8IQ6wlfw1owXWRWw,1C2lxzUo1Hyye4RFIXly3g,BVndHaLihEYbr76Z0CMEGw,5,0,0,0,"Great place for breakfast! I had the waffle, w...",2014-10-11 16:22:06
2,R3TNDNoRUiVfRgvvczy0mg,-Dt5o6GpQcXQfVeWpHNtDg,BVndHaLihEYbr76Z0CMEGw,5,1,1,1,We came here based on the recommendation we re...,2018-04-15 19:51:54


Dividimos date en hora y fecha ya que la hora nos podria servir para analisis

In [14]:
concatenated_df[["date", "time"]] = concatenated_df["date"].str.split(expand=True)
concatenated_df["date"] = pd.to_datetime(concatenated_df["date"])

no hay nulos por lo que no hay problema en realizar la carga

In [16]:
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821793 entries, 0 to 821792
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    821793 non-null  object        
 1   user_id      821793 non-null  object        
 2   business_id  821793 non-null  object        
 3   stars_x      821793 non-null  int64         
 4   useful       821793 non-null  int64         
 5   funny        821793 non-null  int64         
 6   cool         821793 non-null  int64         
 7   text         821793 non-null  object        
 8   date         821793 non-null  datetime64[ns]
 9   time         821793 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 62.7+ MB


In [17]:
concatenated_df.to_csv('/content/drive/MyDrive/Otros/yelp_review_florida.csv', index=False)

**ETL de checkin**

In [18]:
df_checkin= pd.read_json("/content/drive/MyDrive/yelp/Copia de checkin.json", lines=True)

In [19]:
df_checkin.head(3)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"


realizamos el merge con el dataset de business para filtrar solo por florida

In [ ]:
yelp_checkin_florida = df_checkin.merge(df_business, on='business_id', how='inner')

Filtramos por las columnas de checkin

In [ ]:
yelp_checkin_florida = yelp_checkin_florida[df_checkin.columns]

Contamos los check-ins por negocio para evaluar su popularidad y nivel de actividad. Esto nos proporciona información clave sobre la interacción de los clientes con cada negocio y nos ayuda a entender mejor su atractivo y patrones de afluencia

In [ ]:
yelp_checkin_florida['checkin_count'] = yelp_checkin_florida['date'].apply(lambda x: len(x.split(', ')))
yelp_checkin_florida = yelp_checkin_florida[['business_id','checkin_count']]

no hay nulos por lo que no hay problema en realizar la carga

In [ ]:
yelp_checkin_florida.info()

In [ ]:
yelp_checkin_florida.to_csv('/content/drive/MyDrive/Otros/yelp_checkin_florida.csv', index=False)